## Task 1: Finding Similar Items

Randomly select 1000 abstracts from the whole dataset. Find the similar items using pairwise Jaccard similarities, MinHash and LSH (vectorized versions) .

   1. Compare the performance in time and the results for *k*-shingles = 3, 5 and 10, for the three methods and similarity thresholds *s*=0.1 and 0.2. Use 50 hashing functions. Comment your results. 
      
   2. Compare the results obtained for MinHash and LSH for different similarity thresholds *s* = 0.5, 0.9 and 0.95  and 50, 100 and 200 hashing functions. Comment your results.
   
   3. For MinHashing using 100 hashing functions and s = 0.1 and 0.2, find the Jaccard distances (1-Jaccard similarity) for all possible pairs. Use the obtained values within a k-NN algorithm, and for k=1,3 and, 5 identify the clusters with similar abstracts for each s. Describe the obtained clusters, are they different?. Select randomly at least 5 abstracts per cluster, upon visual inspection, what are the main topics?

#    1. Compare the performance in time and the results for *k*-shingles = 3, 5 and 10, for the three methods and similarity thresholds *s*=0.1 and 0.2. Use 50 hashing functions. Comment your results. 

Shingling is not useful while working with high number of files. Because it computes pairwise Jaccard similarities for every pair of docs and it takes lots of time. It took approximately 10 minutes.

Minhashing converts large sets into smaller signatures, while preserving similarity. Hence it takes less time. It took around 10 seconds.

LSH only focuses on pairs of signatures likely to be similar hence it shortens the time even more. It took less than 1 second.

As k-shingles increases, the number of candidates decreases.

In [1]:
# import texts
import pandas as pd
from urllib.request import urlopen
from io import BytesIO
import gzip

url = 'https://aclanthology.org/anthology+abstracts.bib.gz'
with gzip.open(BytesIO(urlopen(url).read()), 'rb') as fb:
    with open('test.bib', 'wb') as f:
        f.write(fb.read())
        
file = open('test.bib')
for n in range(30):
    print(file.readline()[:-1])

@proceedings{woah-2021-online,
    title = "Proceedings of the 5th Workshop on Online Abuse and Harms (WOAH 2021)",
    editor = "Mostafazadeh Davani, Aida  and
      Kiela, Douwe  and
      Lambert, Mathias  and
      Vidgen, Bertie  and
      Prabhakaran, Vinodkumar  and
      Waseem, Zeerak",
    month = aug,
    year = "2021",
    address = "Online",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/2021.woah-1.0",
}
@inproceedings{singh-li-2021-exploiting,
    title = "Exploiting Auxiliary Data for Offensive Language Detection with Bidirectional Transformers",
    author = "Singh, Sumer  and
      Li, Sheng",
    booktitle = "Proceedings of the 5th Workshop on Online Abuse and Harms (WOAH 2021)",
    month = aug,
    year = "2021",
    address = "Online",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/2021.woah-1.1",
    doi = "10.18653/v1/2021.woah-1.1",
    pages = "1--5",
    abstra

In [2]:
# adding abstracts into a list

list_of_abstracts = []
key_word = '    abstract = "'
with open('anthology+abstracts.bib', encoding="utf8") as bibtex_file:
    for line in bibtex_file:
        if line.startswith(key_word):
            list_of_abstracts.append(line[len(key_word):-3])

In [3]:
# first 3 abstracts
list_of_abstracts[:3]

['Offensive language detection (OLD) has received increasing attention due to its societal impact. Recent work shows that bidirectional transformer based methods obtain impressive performance on OLD. However, such methods usually rely on large-scale well-labeled OLD datasets for model training. To address the issue of data/label scarcity in OLD, in this paper, we propose a simple yet effective domain adaptation approach to train bidirectional transformers. Our approach introduces domain adaptation (DA) training procedures to ALBERT, such that it can effectively exploit auxiliary data from source domains to improve the OLD performance in a target domain. Experimental results on benchmark datasets show that our approach, ALBERT (DA), obtains the state-of-the-art performance in most cases. Particularly, our approach significantly benefits underrepresented and under-performing classes, with a significant improvement over ALBERT.',
 'Hate speech and profanity detection suffer from data spar

In [4]:
# number of abstracts
print('Number of abstracts:', len(list_of_abstracts))

Number of abstracts: 30353


In [5]:
# clean up the abstracts
import re
minimum = 200
example = 0
count = {'empty':0, 
         f'short(<{minimum} char)': 0,
         'non latin': 0,
         'other': 0}

abstracts = []


for a in list_of_abstracts:
    if len(a) == 0:
        count['empty'] +=1
    elif len(re.findall('[a-zA-Z]', a)) < .2*len(a):
        count['non latin'] +=1
        if example < 2:
            print(a, '\n')
            example +=1
    elif len(a) < minimum:
        count[f'short(<{minimum} char)'] += 1
    else:
              count['other'] +=1
              abstracts.append(a)
            
for k,v in count.items():
              print(f'{k:>20}', f'{v:>6}')

{``}在汉语等其他有省略代词习惯的语言中,通常会删掉可从上下文信息推断出的代词。尽管以Transformer为代表的的神经机器翻译模型取得了巨大的成功,但这种省略现象依旧对神经机器翻译模型造成了很大的挑战。本文在Transformer基础上提出了一个融合零指代识别的翻译模型,并引入篇章上下文来丰富指代信息。具体地,该模型采用联合学习的框架,在翻译模型基础上,联合了一个分类任务,即判别句子中省略代词在句子所表示的成分,使得模型能够融合零指代信息辅助翻译。通过在中英对话数据集上的实验,验证了本文提出方法的有效性,与基准模型相比,翻译性能提升了1.48个BLEU值。{''} 

{``}机器译文自动评价对机器翻译的发展和应用起着重要的促进作用,它一般通过计算机器译文和人工参考译文的相似度来度量机器译文的质量。该文通过跨语种预训练语言模型XLM将源语言句子、机器译文和人工参考译文映射到相同的语义空间,结合分层注意力和内部注意力提取源语言句子与机器译文、机器译文与人工参考译文以及源语言句子与人工参考译文之间差异特征,并将其融入到基于Bi-LSTM神经译文自动评价方法中。在WMT{'}19译文自动评价数据集上的实验结果表明,融合XLM词语表示的神经机器译文自动评价方法显著提高了其与人工评价的相关性。{''} 

               empty    109
    short(<200 char)     56
           non latin    145
               other  30043


In [6]:
print('Number of abstracts after cleaning it up:', len(abstracts))

Number of abstracts after cleaning it up: 30043


In [7]:
# randomly select 1000 abstracts from the whole cleaned dataset.
import random
abstracts = random.sample(abstracts, 1000)
print('Number of abstracts after selecting randomly:', len(abstracts))

Number of abstracts after selecting randomly: 1000


In [8]:
abstract = abstracts[0]

## Shingling

### Shingling    k:3, s:0.1

In [9]:
import numpy
import os
import re
import binascii
from time import time

def get_shingles(abstract, k=3):
    L = len(abstract)
    shingles = set()          
    for i in range(L-k+1):
        shingles.add(binascii.crc32(abstract[i:i+k].encode('utf-8') ) )           
    return shingles

In [10]:
shingles_vectors = []

for file in abstracts: 
    sh = list(get_shingles(file, k=3))
    shingles_vectors.append(sh)

In [11]:
def jaccard_similarity_score(x, y):
    """
    Jaccard Similarity J (A,B) = | Intersection (A,B) | /
                                    | Union (A,B) |
    """
    intersection_cardinality = len(set(x).intersection(set(y)))
    union_cardinality = len(set(x).union(set(y)))
    return intersection_cardinality / float(union_cardinality)

jaccard_similarity_score(shingles_vectors[0], shingles_vectors[1])

0.2249034749034749

In [12]:
%%time
import itertools

s = 0.1
candidates = []

for pair in itertools.combinations(abstracts,2):
    js = jaccard_similarity_score(get_shingles(pair[0], k=3),get_shingles(pair[1], k=3))
    
    if js > s:
        candidates.append(pair)
        
print(f'Number of candidates: {len(candidates)}')

Number of candidates: 490516
Wall time: 9min 5s


### Shingling k:3, s:0.2

In [13]:
%%time
s = 0.2
candidates = []

for pair in itertools.combinations(abstracts,2):
    js = jaccard_similarity_score(get_shingles(pair[0], k=3),get_shingles(pair[1], k=3))
    
    if js > s:
        candidates.append(pair)
        
print(f'Number of candidates: {len(candidates)}')

Number of candidates: 290504
Wall time: 9min 26s


### Shingling  k:5, s:0.1

In [14]:
def get_shingles(abstract, k=5):
    L = len(abstract)
    shingles = set()          
    for i in range(L-k+1):
        shingles.add(binascii.crc32(abstract[i:i+k].encode('utf-8') ) )           
    return shingles

In [15]:
shingles_vectors = []

for file in abstracts: 
    sh = list(get_shingles(file, k=5))
    shingles_vectors.append(sh)

In [16]:
%%time

s = 0.1
candidates = []

for pair in itertools.combinations(abstracts,2):
    js = jaccard_similarity_score(get_shingles(pair[0], k=5),get_shingles(pair[1], k=5))
    
    if js > s:
        candidates.append(pair)
        
print(f'Number of candidates: {len(candidates)}')

Number of candidates: 3212
Wall time: 10min 29s


### Shingling k:5, s:0.2

In [17]:
%%time

s = 0.2
candidates = []

for pair in itertools.combinations(abstracts,2):
    js = jaccard_similarity_score(get_shingles(pair[0], k=5),get_shingles(pair[1], k=5))
    
    if js > s:
        candidates.append(pair)
        
print(f'Number of candidates: {len(candidates)}')

Number of candidates: 2
Wall time: 11min 16s


### Shingling  k:10, s:0.1

In [18]:
def get_shingles(abstract, k=10):
    L = len(abstract)
    shingles = set()          
    for i in range(L-k+1):
        shingles.add(binascii.crc32(abstract[i:i+k].encode('utf-8') ) )           
    return shingles

In [19]:
shingles_vectors = []

for file in abstracts: 
    sh = list(get_shingles(file, k=10))
    shingles_vectors.append(sh)

In [20]:
%%time
import itertools

s = 0.1
candidates = []

for pair in itertools.combinations(abstracts,2):
    js = jaccard_similarity_score(get_shingles(pair[0], k=10),get_shingles(pair[1], k=10))
    
    if js > s:
        candidates.append(pair)
        
print(f'Number of candidates: {len(candidates)}')

Number of candidates: 1
Wall time: 11min 2s


### Shingling k:10, s:0.2

In [21]:
%%time

s = 0.2
candidates = []

for pair in itertools.combinations(abstracts,2):
    js = jaccard_similarity_score(get_shingles(pair[0], k=10),get_shingles(pair[1], k=10))
    
    if js > s:
        candidates.append(pair)
        
print(f'Number of candidates: {len(candidates)}')

Number of candidates: 1
Wall time: 11min 35s


## Minhashing

### Minhashing  k:5, s:0.1

In [22]:
def get_shingles(abstract, k=5):
    L = len(abstract)
    shingles = set()          
    for i in range(L-k+1):
        shingles.add(binascii.crc32(abstract[i:i+k].encode('utf-8') ) )           
    return shingles

In [23]:
print("file:{}".format(abstracts[0]))
print("number of shingles: {}".format(len(get_shingles(abstracts[0], k=5))))

file:We propose a novel tensor embedding method that can effectively extract lexical features for humor recognition. Specifically, we use word-word co-occurrence to encode the contextual content of documents, and then decompose the tensor to get corresponding vector representations. We show that this simple method can capture features of lexical humor effectively for continuous humor recognition. In particular, we achieve a distance of 0.887 on a global humor ranking task, comparable to the top performing systems from SemEval 2017 Task 6B (Potash et al., 2017) but without the need for any external training corpus. In addition, we further show that this approach is also beneficial for small sample humor recognition tasks through a semi-supervised label propagation procedure, which achieves about 0.7 accuracy on the 16000 One-Liners (Mihalcea and Strapparava, 2005) and Pun of the Day (Yang et al., 2015) humour classification datasets using only 10{\%} of known labels.
number of shingles:

In [24]:
# set global parameters to process the whole dataset
bands = 10
rows = 5
nsig = bands*rows  # number of elements in signature, or the number of different random hash functions

maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)
B = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)

In [25]:
print(len(A),A)

50 [2608428423 2625020704  317044267 1726195090 4058616660 2280834150
  642999764 1865178151 2934700102 3325181062 1874789733 1839672909
 3498069159  408492336   90301516 1431956758 2729065828  136935333
 1552452541 1794071519 1212668039    2943883 3633368108 2465656604
  519014756 2112079809 2891052589 1563307481 4121614587  726915284
  983608387 1863092456 2348804910 2495972336 1014574098  582140236
  873127908  210182330 1908036527 4156881913 4082800064 1535204616
 2398712886    2206787 1631686503 3482275748 1129636301 2108977863
  421860581 2969324923]


In [26]:
ShingleID = list(get_shingles(abstract, k=5))[0]

print("random shingle: {}".format(ShingleID))

hashCode = ((A[0]*ShingleID + B[0]) % nextPrime) % maxShingleID
print("its hash code by first hash function: {}".format(hashCode))
hashCode = ((A[1]*ShingleID + B[1]) % nextPrime) % maxShingleID
print("its hash code by second hash function: {}".format(hashCode))

random shingle: 1619480577
its hash code by first hash function: 3933608767
its hash code by second hash function: 3135531263


In [27]:
# naive version of Minhash algorithm that computes a signature for a single file
# all shingles from that file are given in 'shingles'

def minhash(shingles, A, B, nextPrime, maxShingleID, nsig):
    signature = []
    for i in range(nsig):  # number of hash functions == nsig
        minHashCode = maxShingleID + 1
        a = A[i]
        b = B[i]
        
        for ShingleID in shingles:
            hashCode = ((a*ShingleID + b) % nextPrime) % maxShingleID
            if hashCode < minHashCode:
                minHashCode = hashCode

        signature.append(minHashCode)
    return signature

In [28]:
shingles = get_shingles(abstract, k=5)
maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A = numpy.random.randint(0, nextPrime/2, size=(nsig,), dtype = numpy.int64)
B = numpy.random.randint(0, nextPrime/2, size=(nsig,), dtype = numpy.int64)

signature = minhash(shingles, A, B, nextPrime, maxShingleID, nsig)
print("file signature: {} , {}".format(len(signature),signature))

file signature: 50 , [5318889, 1649361, 768091, 2509403, 7600308, 13947042, 614913, 5650378, 8679378, 4864527, 5159907, 1240308, 5303800, 7347425, 3801393, 3426982, 9990741, 1245743, 3123, 6614762, 4818795, 5606449, 244496, 6248275, 1897902, 9215178, 2363666, 4405532, 5820443, 371064, 2914492, 5365917, 2761972, 2767852, 2236010, 2029690, 446360, 10616707, 2468366, 5894004, 20473406, 2785576, 5199159, 1245919, 4880741, 1561817, 7936019, 7816122, 1454999, 4022773]


In [29]:
# compute Minhashes for all files using a slow naive code
signatures = []
t = time()
for fname in abstracts:
    shingles = get_shingles(fname, k=5)
    signature = minhash(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)
t1 = time()-t
print("total signatures: {}".format(len(signatures)))
print("took {} seconds".format(t1))

total signatures: 1000
took 34.82668113708496 seconds


In [30]:
# fast implementation of Minhash algorithm
# computes all random hash functions for a shingle at once, using vector operations
# also finds element-wise minimum of two vectors efficiently
def minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig):
    signature = numpy.ones((nsig,)) * (maxShingleID + 1)

    for ShingleID in shingles:
        hashCodes = ((A*ShingleID + B) % nextPrime) % maxShingleID
        numpy.minimum(signature, hashCodes, out=signature)

    return signature

In [31]:
# compare two versions of Minhash code
shingles_all_files = []
for fname in abstracts:
    shingles_all_files.append(get_shingles(fname, k=5))

t = time()
signatures_all_files_1 = []
for shingles in shingles_all_files:
    signature = minhash(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures_all_files_1.append(signature)
t1 = time()-t
print("slow code took {} seconds".format(t1))

t = time()
signatures_all_files_2 = []
for shingles in shingles_all_files:
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures_all_files_2.append(signature)
t2 = time()-t
print("really fast code took {} seconds".format(t2))

print('speedup {}'.format(t1/t2))

signatures_all_files_1 = numpy.array(signatures_all_files_1)
signatures_all_files_2 = numpy.array(signatures_all_files_2)
print("results are the same: {}".format(numpy.allclose(signatures_all_files_1, signatures_all_files_2)))

slow code took 35.158782720565796 seconds
really fast code took 4.634079694747925 seconds
speedup 7.587004332362543
results are the same: True


In [32]:
# get candidate pairs without Locality-Sensitive Hashing

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=5)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

s = 0.1  # similarity threshold
Nfiles = len(signatures)
t = time()
candidates = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
        if Jsim >= s:                                      # two vectors, equivalente to Jaccard 
            candidates.append((i,j))
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 5.7157392501831055 seconds
found 35318 candidates


### Minhashing  k:5, s:0.2

In [33]:
# get candidate pairs without Locality-Sensitive Hashing

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=5)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

s = 0.2  # similarity threshold
Nfiles = len(signatures)
t = time()
candidates = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
        if Jsim >= s:                                      # two vectors, equivalente to Jaccard 
            candidates.append((i,j))
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 5.965559482574463 seconds
found 158 candidates


### Minhashing k:3, s:0.1

In [34]:
def get_shingles(abstract, k=3):
    L = len(abstract)
    shingles = set()          
    for i in range(L-k+1):
        shingles.add(binascii.crc32(abstract[i:i+k].encode('utf-8') ) )           
    return shingles

In [35]:
ShingleID = list(get_shingles(abstract, k=3))[0]

print("random shingle: {}".format(ShingleID))

hashCode = ((A[0]*ShingleID + B[0]) % nextPrime) % maxShingleID
print("its hash code by first hash function: {}".format(hashCode))
hashCode = ((A[1]*ShingleID + B[1]) % nextPrime) % maxShingleID
print("its hash code by second hash function: {}".format(hashCode))

random shingle: 3230435331
its hash code by first hash function: 712734687
its hash code by second hash function: 2517094247


In [36]:
shingles_all_files = []
for fname in abstracts:
    shingles_all_files.append(get_shingles(fname, k=3))

t = time()
signatures_all_files_2 = []
for shingles in shingles_all_files:
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures_all_files_2.append(signature)
t2 = time()-t
print("really fast code took {} seconds".format(t2))

really fast code took 3.2175002098083496 seconds


In [37]:
# get candidate pairs without Locality-Sensitive Hashing

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=3)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

s = 0.1  # similarity threshold
Nfiles = len(signatures)
t = time()
candidates = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
        if Jsim >= s:                                      # two vectors, equivalente to Jaccard 
            candidates.append((i,j))
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 5.830845832824707 seconds
found 442783 candidates


### Minhashing  k:3, s:0.2

In [38]:
# get candidate pairs without Locality-Sensitive Hashing

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=3)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

s = 0.2  # similarity threshold
Nfiles = len(signatures)
t = time()
candidates = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
        if Jsim >= s:                                      # two vectors, equivalente to Jaccard 
            candidates.append((i,j))
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 5.815581321716309 seconds
found 162586 candidates


### Minhashing k:10, s:0.1

In [39]:
def get_shingles(abstract, k=10):
    L = len(abstract)
    shingles = set()          
    for i in range(L-k+1):
        shingles.add(binascii.crc32(abstract[i:i+k].encode('utf-8') ) )           
    return shingles

In [40]:
ShingleID = list(get_shingles(abstract, k=10))[0]

print("random shingle: {}".format(ShingleID))

hashCode = ((A[0]*ShingleID + B[0]) % nextPrime) % maxShingleID
print("its hash code by first hash function: {}".format(hashCode))
hashCode = ((A[1]*ShingleID + B[1]) % nextPrime) % maxShingleID
print("its hash code by second hash function: {}".format(hashCode))

random shingle: 1335128064
its hash code by first hash function: 621178495
its hash code by second hash function: 3207715175


In [41]:
shingles_all_files = []
for fname in abstracts:
    shingles_all_files.append(get_shingles(fname, k=10))

t = time()
signatures_all_files_2 = []
for shingles in shingles_all_files:
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures_all_files_2.append(signature)
t2 = time()-t
print("really fast code took {} seconds".format(t2))

really fast code took 5.33053731918335 seconds


In [42]:
# get candidate pairs without Locality-Sensitive Hashing

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=10)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

s = 0.1  # similarity threshold
Nfiles = len(signatures)
t = time()
candidates = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
        if Jsim >= s:                                      # two vectors, equivalente to Jaccard 
            candidates.append((i,j))
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 6.058567047119141 seconds
found 108 candidates


### Minhashing - k:10, s:0.2

In [43]:
# get candidate pairs without Locality-Sensitive Hashing

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=10)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

s = 0.2  # similarity threshold
Nfiles = len(signatures)
t = time()
candidates = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
        if Jsim >= s:                                      # two vectors, equivalente to Jaccard 
            candidates.append((i,j))
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 5.747239351272583 seconds
found 1 candidates


### LSH - k:3

In [44]:
def LSH(signatures, bands, rows, Ab, Bb, nextPrime, maxShingleID):
    """Locality Sensitive Hashing
    """
    numItems = signatures.shape[1]
    signBands = numpy.array_split(signatures, bands, axis=0)
    candidates = set()
    for nb in range(bands):
        hashTable = {}
        for ni in range(numItems):
            item = signBands[nb][:,ni]
            hash = (numpy.dot(Ab[nb,:], item) + Bb[nb]) % nextPrime % maxShingleID
            if hash not in hashTable:
                hashTable[hash] = [ni]
            else:
                hashTable[hash].append(ni)
        for _,items in hashTable.items():
            if len(items) > 1:
                L = len(items)
                for i in range(L-1):
                    for j in range(i+1, L):
                        cand = [items[i], items[j]]
                        numpy.sort(cand)
                        candidates.add(tuple(cand))
    return candidates



# set global parameters to process the whole dataset
bands = 10
rows = 5
nsig = bands*rows  # number of elements in signature, or the number of different random hash functions

maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)
B = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)

In [45]:
# find candidates with LSH

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=3)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

# prepare data for LSH
A2 = numpy.random.randint(0, nextPrime/2, size=(bands, rows), dtype = numpy.int64)  # now we need a vector of A parameters for each band
B2 = numpy.random.randint(0, nextPrime/2, size=(bands, ), dtype = numpy.int64)
signatures = numpy.array(signatures).T  # LSH needs a matrix of signatures, not a list of vectors

#s = 0.95  # NO similarity threshold, why?
Nfiles = signatures.shape[1]  # number of different files
t = time()
candidates = LSH(signatures, bands, rows, A2, B2, nextPrime, maxShingleID)
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 0.0995032787322998 seconds
found 3638 candidates


### LSH  k:5

In [46]:
# find candidates with LSH

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=5)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

# prepare data for LSH
A2 = numpy.random.randint(0, nextPrime/2, size=(bands, rows), dtype = numpy.int64)  # now we need a vector of A parameters for each band
B2 = numpy.random.randint(0, nextPrime/2, size=(bands, ), dtype = numpy.int64)
signatures = numpy.array(signatures).T  # LSH needs a matrix of signatures, not a list of vectors

#s = 0.95  # NO similarity threshold, why?
Nfiles = signatures.shape[1]  # number of different files
t = time()
candidates = LSH(signatures, bands, rows, A2, B2, nextPrime, maxShingleID)
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 0.05013537406921387 seconds
found 4 candidates


### LSH  k:10

In [47]:
# find candidates with LSH

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=10)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

# prepare data for LSH
A2 = numpy.random.randint(0, nextPrime/2, size=(bands, rows), dtype = numpy.int64)  # now we need a vector of A parameters for each band
B2 = numpy.random.randint(0, nextPrime/2, size=(bands, ), dtype = numpy.int64)
signatures = numpy.array(signatures).T  # LSH needs a matrix of signatures, not a list of vectors

#s = 0.95  # NO similarity threshold, why?
Nfiles = signatures.shape[1]  # number of different files
t = time()
candidates = LSH(signatures, bands, rows, A2, B2, nextPrime, maxShingleID)
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 0.03135251998901367 seconds
found 0 candidates


# 2. Compare the results obtained for MinHash and LSH for different similarity thresholds s = 0.1, 0.2 and 0.25 and 50, 100 and 200 hashing functions. Comment your results.

As hashing fuctions increases, the number of candidates decreases. 

LSH is very fast, it takes less than 1 second. And by using LSH I found significantly less number of candidates especially comparing minhashing with s = 0.1 similarity threshold.

### Minhash - s: 0.1, hashing:50

In [48]:
def get_shingles(abstract, k=5):
    L = len(abstract)
    shingles = set()          
    for i in range(L-k+1):
        shingles.add(binascii.crc32(abstract[i:i+k].encode('utf-8') ) )           
    return shingles

In [49]:
# set global parameters to process the whole dataset
bands = 10
rows = 5
nsig = bands*rows  # number of elements in signature, or the number of different random hash functions

maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)
B = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)

In [50]:
ShingleID = list(get_shingles(abstract, k=5))[0]

print("random shingle: {}".format(ShingleID))

hashCode = ((A[0]*ShingleID + B[0]) % nextPrime) % maxShingleID
print("its hash code by first hash function: {}".format(hashCode))
hashCode = ((A[1]*ShingleID + B[1]) % nextPrime) % maxShingleID
print("its hash code by second hash function: {}".format(hashCode))

random shingle: 1619480577
its hash code by first hash function: 4078442722
its hash code by second hash function: 1490076031


In [51]:
shingles_all_files = []
for fname in abstracts:
    shingles_all_files.append(get_shingles(fname, k=5))

t = time()
signatures_all_files_2 = []
for shingles in shingles_all_files:
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures_all_files_2.append(signature)
t2 = time()-t
print("really fast code took {} seconds".format(t2))

really fast code took 4.582310914993286 seconds


In [52]:
# get candidate pairs without Locality-Sensitive Hashing

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=5)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

s = 0.1  # similarity threshold
Nfiles = len(signatures)
t = time()
candidates = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
        if Jsim >= s:                                      # two vectors, equivalente to Jaccard 
            candidates.append((i,j))
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 5.636272430419922 seconds
found 107813 candidates


### Minhash - s:0.2, hashing:50

In [53]:
s = 0.2  # similarity threshold
Nfiles = len(signatures)
t = time()
candidates = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
        if Jsim >= s:                                      # two vectors, equivalente to Jaccard 
            candidates.append((i,j))
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 5.84703516960144 seconds
found 1481 candidates


### Minhash - s:0.25, hashing:50

In [54]:
s = 0.25  # similarity threshold
Nfiles = len(signatures)
t = time()
candidates = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
        if Jsim >= s:                                      # two vectors, equivalente to Jaccard 
            candidates.append((i,j))
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 5.9132020473480225 seconds
found 41 candidates


### Minhash - s:0.1, hashing:100

In [55]:
def get_shingles(abstract, k=5):
    L = len(abstract)
    shingles = set()          
    for i in range(L-k+1):
        shingles.add(binascii.crc32(abstract[i:i+k].encode('utf-8') ) )           
    return shingles

In [56]:
# set global parameters to process the whole dataset
bands = 10
rows = 10
nsig = bands*rows  # number of elements in signature, or the number of different random hash functions

maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)
B = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)

In [57]:
# get candidate pairs without Locality-Sensitive Hashing

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=5)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

s = 0.1  # similarity threshold
Nfiles = len(signatures)
t = time()
candidates = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
        if Jsim >= s:                                      # two vectors, equivalente to Jaccard 
            candidates.append((i,j))
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 6.0353102684021 seconds
found 36034 candidates


### Minhash - s:0.2, hashing: 100

In [58]:
# get candidate pairs without Locality-Sensitive Hashing

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=5)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

s = 0.2  # similarity threshold
Nfiles = len(signatures)
t = time()
candidates = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
        if Jsim >= s:                                      # two vectors, equivalente to Jaccard 
            candidates.append((i,j))
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 6.065131187438965 seconds
found 34 candidates


### Minhash - s:0.25, hashing: 100

In [59]:
# get candidate pairs without Locality-Sensitive Hashing

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=5)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

s = 0.25  # similarity threshold
Nfiles = len(signatures)
t = time()
candidates = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
        if Jsim >= s:                                      # two vectors, equivalente to Jaccard 
            candidates.append((i,j))
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 6.364579439163208 seconds
found 2 candidates


### Minhash - s:0.1, hashing: 200

In [60]:
# set global parameters to process the whole dataset
bands = 20
rows = 10
nsig = bands*rows  # number of elements in signature, or the number of different random hash functions

maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)
B = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)

In [61]:
# get candidate pairs without Locality-Sensitive Hashing

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=5)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

s = 0.1  # similarity threshold
Nfiles = len(signatures)
t = time()
candidates = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
        if Jsim >= s:                                      # two vectors, equivalente to Jaccard 
            candidates.append((i,j))
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 6.3162922859191895 seconds
found 17875 candidates


### Minhash - s:0.2, hashing: 200

In [62]:
# get candidate pairs without Locality-Sensitive Hashing

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=5)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

s = 0.2  # similarity threshold
Nfiles = len(signatures)
t = time()
candidates = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
        if Jsim >= s:                                      # two vectors, equivalente to Jaccard 
            candidates.append((i,j))
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 6.480101585388184 seconds
found 3 candidates


### Minhash - s:0.25, hashing: 200

In [63]:
# get candidate pairs without Locality-Sensitive Hashing

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=5)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

s = 0.25  # similarity threshold
Nfiles = len(signatures)
t = time()
candidates = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
        if Jsim >= s:                                      # two vectors, equivalente to Jaccard 
            candidates.append((i,j))
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 6.232314109802246 seconds
found 1 candidates


### LSH 50 hashing

In [64]:
# set global parameters to process the whole dataset
bands = 10
rows = 5
nsig = bands*rows  # number of elements in signature, or the number of different random hash functions

maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)
B = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)

# find candidates with LSH

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=5)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

# prepare data for LSH
A2 = numpy.random.randint(0, nextPrime/2, size=(bands, rows), dtype = numpy.int64)  # now we need a vector of A parameters for each band
B2 = numpy.random.randint(0, nextPrime/2, size=(bands, ), dtype = numpy.int64)
signatures = numpy.array(signatures).T  # LSH needs a matrix of signatures, not a list of vectors

#s = 0.95  # NO similarity threshold, why?
Nfiles = signatures.shape[1]  # number of different files
t = time()
candidates = LSH(signatures, bands, rows, A2, B2, nextPrime, maxShingleID)
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 0.05684399604797363 seconds
found 6 candidates


### LSH 100 hashing

In [65]:
# set global parameters to process the whole dataset
bands = 10
rows = 10
nsig = bands*rows  # number of elements in signature, or the number of different random hash functions

maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)
B = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)

# find candidates with LSH

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=5)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

# prepare data for LSH
A2 = numpy.random.randint(0, nextPrime/2, size=(bands, rows), dtype = numpy.int64)  # now we need a vector of A parameters for each band
B2 = numpy.random.randint(0, nextPrime/2, size=(bands, ), dtype = numpy.int64)
signatures = numpy.array(signatures).T  # LSH needs a matrix of signatures, not a list of vectors

#s = 0.95  # NO similarity threshold, why?
Nfiles = signatures.shape[1]  # number of different files
t = time()
candidates = LSH(signatures, bands, rows, A2, B2, nextPrime, maxShingleID)
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 0.05050373077392578 seconds
found 0 candidates


### LSH 200 hashing

In [66]:
# set global parameters to process the whole dataset
bands = 10
rows = 20
nsig = bands*rows  # number of elements in signature, or the number of different random hash functions

maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)
B = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)

# find candidates with LSH

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=5)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

# prepare data for LSH
A2 = numpy.random.randint(0, nextPrime/2, size=(bands, rows), dtype = numpy.int64)  # now we need a vector of A parameters for each band
B2 = numpy.random.randint(0, nextPrime/2, size=(bands, ), dtype = numpy.int64)
signatures = numpy.array(signatures).T  # LSH needs a matrix of signatures, not a list of vectors

Nfiles = signatures.shape[1]  # number of different files
t = time()
candidates = LSH(signatures, bands, rows, A2, B2, nextPrime, maxShingleID)
t2 = time() - t
print("finding candidates took {} seconds".format(t2))
print("found {} candidates".format(len(candidates)))

finding candidates took 0.04929041862487793 seconds
found 0 candidates


## 3. For MinHashing using 100 hashing functions and s = 0.1 and 0.2, find the Jaccard distances (1-Jaccard similarity) for all possible pairs. Use the obtained values within a k-NN algorithm, and for k=1,3 and, 5 identify the clusters with similar abstracts for each s. Describe the obtained clusters, are they different?. Select randomly at least 5 abstracts per cluster, upon visual inspection, what are the main topics?

In [67]:
# set global parameters to process the whole dataset
bands = 10
rows = 10
nsig = bands*rows  # number of elements in signature, or the number of different random hash functions

maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)
B = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)

In [68]:
# get candidate pairs without Locality-Sensitive Hashing

signatures = []  # signatures for all files

for fname in abstracts:
    shingles = get_shingles(fname, k=5)
    signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(signature)

s = 0.1  # similarity threshold
Nfiles = len(signatures)
t = time()
candidates = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
        if Jsim >= s:                                      # two vectors, equivalente to Jaccard 
            candidates.append(Jsim)

In [82]:
# getting jaccard distances by using 1 - jaccard similarity

distances = [1 - x for x in candidates]